# League of legend Winner team predict to Using Riot API

2019 데이터분석 개인 프로젝트 리메이크

리그 오브 레전드 경기 기록을 수집하고, 가공하여 승률에 영향을 주는 요인 분석과 실시간 스코어 변동에 따른 승률 예측 프로젝트

A project to collect league-of-legend game records to analyze the factors that affect victory and to predict winning rates based on real-time score fluctuations.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from tqdm import tqdm_notebook # 오래 걸리는 작업 진행확인용
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
match = pd.read_pickle("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/matchpre.pkl")

win_team_stat = pd.read_csv("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/win_team_stats.csv")

lose_team_stat = pd.read_csv("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/lose_team_stats.csv")

win_team = pd.read_pickle("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/match_winner_data.pkl")

lose_team = pd.read_pickle("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/match_lose_data.pkl")

date = pd.read_csv("/kaggle/input/lol-classic-rank-game-datakrtop-3-tier/lol_version_Date.csv")

In [ ]:
"""
팀 스탯 row 와 팀 기록 row 일치시키기
"""
gameId = win_team_stat["gameId"]
match = pd.merge(gameId,match,how="inner",on="gameId")
win_team = pd.merge(gameId,win_team,how="inner",on="gameId")
lose_team = pd.merge(gameId,lose_team,how="inner",on="gameId")

"""
duplicated() 함수는 리스트에 대한 중복검사를 지원하지 않으므로 리스트 칼럼 삭제
"""
match.drop("participants",axis=1,inplace=True)

win_team.drop("bans",axis=1,inplace=True)

lose_team.drop("bans",axis=1,inplace=True)

match = match.drop_duplicates()
win_team = win_team.drop_duplicates()
lose_team = lose_team.drop_duplicates()

win_team_stat = win_team_stat.drop_duplicates()
lose_team_stat = lose_team_stat.drop_duplicates()

"""
win_team, lose_team으로 나누어 모든 테이블 병합
"""
win_team = pd.merge(match,win_team,how="left",on="gameId")
win_team = pd.merge(win_team,win_team_stat,how="left",on="gameId")

lose_team = pd.merge(match,lose_team,how="left",on="gameId")
lose_team = pd.merge(lose_team,lose_team_stat,how="left",on="gameId")

del match
gc.collect()

"""
팀 데이터셋을 전체 데이터로 병합하기 전에 칼럼명 일치시키기
"""
win_team.columns = win_team.columns.str.replace("win_","")

lose_team.columns = lose_team.columns.str.replace("lose_","")

"""
전체 게임 정보 gamedata 테이블 생성
"""
gamedata = pd.concat([win_team,lose_team])
gamedata = gamedata.reset_index()
gamedata.drop("index",axis=1,inplace=True)
gamedata = gamedata.astype({"gameVersion":int})
gamedata = pd.merge(gamedata,date,how="inner",on="gameVersion")

del win_team
del lose_team
gc.collect()

"""
카테고리형 데이터(True,False) Label encoding / 분석에 용이하도록 가공
"""
bool_mapping = {True:1,False:0}
bool_col = gamedata.select_dtypes('bool').columns.tolist()

for col in bool_col:
    gamedata[col] = gamedata[col].map(bool_mapping)
    
win_mapping = {"Win":1,"Fail":0}
gamedata["win"] = gamedata["win"].map(win_mapping)

gamedata["date"] = pd.to_datetime(gamedata["date"])
gamedata["gameId"] = gamedata.astype({"gameId":object})

In [ ]:
gamedata

# 4. Feature Engineering / 특성 공학

In [ ]:
#예측에 필요없는 칼럼 드랍(승패예측에 영향을 끼치지 못함)
gamedata.drop(["gameVersion","vilemawKills","dominionVictoryScore","date"],axis=1,inplace=True)

In [ ]:
pd.set_option('display.float_format', '{:.5f}'.format) # 항상 float 형식으로

In [ ]:
#승패에 영향이 가는 상관계수 분석
gamedata.corr()["win"].sort_values()

In [ ]:
gamedata.head()

대체적으로 개인 기록보다 팀 단위 기록들이 승패예측에 더 도움이 된다.

gameDuration 는 게임시간으로 각 변수들 간의 상관관계는 가장 컷지만, 각 경기 시간당 승패가 무조건 존재하기 때문에 단순한 시간만으로는 모델이 게임 시간에 대한 정보를 전혀 습득하지 못할 것이다.

## 개인 기록 팀 단위 변환 - 교호작용(Interaction)

개인 기록은 1-5번의 총 다섯명 유저가 존재한다. 이들의 기록을 합산하여 팀 단위의 기록을 추가해준다.

In [ ]:
#팀의 킬카운트 총합
gamedata["team_kills"] = gamedata["kills1"] + gamedata["kills2"] + gamedata["kills3"] + gamedata["kills4"] + gamedata["kills5"]

#팀의 데스카운트 총합
gamedata["team_deaths"] = gamedata["deaths1"] + gamedata["deaths2"] + gamedata["deaths3"] + gamedata["deaths4"] + gamedata["deaths5"]

#팀이 획득한 총 골드 - 제외됨
#gamedata["team_goldEarned"] = gamedata["goldEarned1"] + gamedata["goldEarned2"] + gamedata["goldEarned3"] + gamedata["goldEarned4"] + gamedata["goldEarned5"]

#팀이 가한 총 피해량
gamedata["team_totalDamageDealtToChampions"] = gamedata["totalDamageDealtToChampions1"] + gamedata["totalDamageDealtToChampions2"] + gamedata["totalDamageDealtToChampions3"] + gamedata["totalDamageDealtToChampions4"] + gamedata["totalDamageDealtToChampions5"]

#팀이 가한 총 CC기 시간
gamedata["team_totalTimeCrowdControlDealt"] = gamedata["totalTimeCrowdControlDealt1"] + gamedata["totalTimeCrowdControlDealt2"] + gamedata["totalTimeCrowdControlDealt3"] + gamedata["totalTimeCrowdControlDealt4"] + gamedata["totalTimeCrowdControlDealt5"]

#팀의 총 시야점수
gamedata["team_visionScore"] = gamedata["visionScore1"] + gamedata["visionScore2"] + gamedata["visionScore3"] + gamedata["visionScore4"] + gamedata["visionScore5"]

#팀이 처치한 총 오브젝트 갯수 - 제외됨
#gamedata["team_Object"] = (gamedata["riftHeraldKills"] + gamedata["baronKills"] + gamedata["dragonKills"] + gamedata["inhibitorKills"] + gamedata["towerKills"])

In [ ]:
#천상계는 전체적으로 하위 티어보다 킬, 데스의 분포 범위가 다름을 고려, 킬/데스의 비율로 킬데스를 반영.
def kdc(df):
    if df["team_deaths"]==0:
        return df["team_kills"]/(df["team_deaths"]+1)*1.2 #만약 팀의 총 데스가 0일경우 퍼펙트 게임을 적용해 가중치 1.2 적용
    return df["team_kills"]/df["team_deaths"]

In [ ]:
#팀의 킬/데스 지표
gamedata["team_K/D"] = gamedata.apply(kdc,axis=1)

In [ ]:
#승패에 영향이 가는 상관계수 분석
gamedata.corr()["win"].sort_values()

새로 생성한 모든 팀 단위 기록이 개인 기록일때보다 상관계수가 높아졌다.

## 팀별 차이 칼럼(반영 안함) - 도메인 지식 활용

>다중공선성 문제와 너무 상관계수가 커서 삭제 반영 안하기로 결정.

리그 오브 레전드의 프로 리그에서 팀의 우위를 정하는 지표들을 활용, 
1. 골드 차이

2. 딜량 차이

In [ ]:
"""
차이 칼럼들은 다른 변수들과 다중공선성 문제가 크게 생기고 회귀결과에 미치는 영향이 너무 커서 제외함.
"""

# from sklearn.preprocessing import LabelEncoder
# gamedata["gameId"] = LabelEncoder().fit_transform(gamedata["gameId"])

# #gameId, win을 기준으로 데이터프레임화
# gamedata = gamedata.sort_values(by=["gameId","win"])

# gamedata_win = gamedata[1::2]

# gamedata_lose = gamedata[::2]

# gamedata_win.reset_index(drop=True,inplace=True)

# gamedata_lose.reset_index(drop=True,inplace=True)

# #승리팀, 진팀의 골드 차이
# gamedata_win["team_goldDiff"] = gamedata_win["team_goldEarned"] - gamedata_lose["team_goldEarned"]
# gamedata_lose["team_goldDiff"] = gamedata_lose["team_goldEarned"] - gamedata_win["team_goldEarned"]

# #승리팀, 진팀의, 딜량 차이
# gamedata_win["team_dealtDiff"] = gamedata_win["team_totalDamageDealtToChampions"] - gamedata_lose["team_totalDamageDealtToChampions"]
# gamedata_lose["team_dealtDiff"] = gamedata_lose["team_totalDamageDealtToChampions"] - gamedata_win["team_totalDamageDealtToChampions"]

# gamedata = pd.concat([gamedata_win,gamedata_lose],axis=0)

# gamedata.reset_index(drop=True,inplace=True)

# #승패에 영향이 가는 상관계수 분석
# gamedata.corr()["win"].sort_values()

도메인 지식을 활용해 얻은 차이 특성들은 굉장히 높은 상관관계를 보인다.

## 게임 시간별 구간 분할 - 이산화(bining)

게임 시간별로 구간을 나눠 시간대가 실제로 변수에 영향을 미치는가 검사

6개의 구간에 해당하는 특성들의 반영도(가중치)가 각각 달라야 하므로 6개의 데이터셋 실험.

(단일 모델로는 게임의 구간별 특성 반영을 조절할 수 없음)

* 6개 구간 중 (0-15, 15-20) (30-35, 35-inf) 구간이 같은 양상을 띄어 4개 구간으로 변경

In [ ]:
#게임시간(초) -> 분
gamedata["gameMinute"] = gamedata["gameDuration"] / 60

#다시하기 경기 제외
gamedata = gamedata[gamedata["gameMinute"] > 5]

In [ ]:
#bins = [0, ,15, 20, 25, 30, 35]
bins = [0, 20, 25, 30]

[0-20), [20-25), [25-30), [30-inf)

[ , ]는 포함, (, )은 미포함을 나타내는 총 4개 구간을 생성

In [ ]:
gamedata["time_bin"] = np.digitize(gamedata["gameMinute"],bins)
gamedata["time_bin"].value_counts()

* 데이터를 4구간으로 변경, 데이터셋의 분포가 일정하게 잘 나뉨.

In [ ]:
game_part1 = gamedata[gamedata["time_bin"] == 1]
game_part2 = gamedata[gamedata["time_bin"] == 2]
game_part3 = gamedata[gamedata["time_bin"] == 3]
game_part4 = gamedata[gamedata["time_bin"] == 4]

game_part1.drop(["gameMinute","gameDuration"],axis=1,inplace=True)
game_part2.drop(["gameMinute","gameDuration"],axis=1,inplace=True)
game_part3.drop(["gameMinute","gameDuration"],axis=1,inplace=True)
game_part4.drop(["gameMinute","gameDuration"],axis=1,inplace=True)

In [ ]:
display(game_part1.corr()["win"].sort_values(),

game_part2.corr()["win"].sort_values(),

game_part3.corr()["win"].sort_values(),

game_part4.corr()["win"].sort_values())

실제로 각 시간대마다 중요한 변수가 다르다는 것을 확인하였다. 또한 게임이 길어질수록 지표들의 영향도가 하락하고 상관계수 간의 차이가 줄어드는 것이 확인된다.

## 게임 시간 조합 - 교호작용(Interaction)

총합이 중요한 것이 아니라 시간당 지표의 변화가 중요한 것이기 때문에 

게임 시간을 이전에 만든 총합 지표와 조합하여 분당 팀의 성장률을 나타내는 지표를 만든다.

In [ ]:
#분당 팀의 킬,데스 스코어

gamedata["team_kills_per_minute"] = gamedata["team_kills"] / gamedata["gameMinute"]

gamedata["team_deaths_per_minute"]= gamedata["team_deaths"] / gamedata["gameMinute"]

gamedata["team_K/D_per_minute"] = gamedata["team_K/D"] / gamedata["gameMinute"]

#분당 팀이 가한 총 데미지
gamedata["team_totalDamageDealt_per_minute"] = gamedata["team_totalDamageDealtToChampions"] / gamedata["gameMinute"]

#분당 팀이 가한 CC기 시간
gamedata["team_totalTimeCrowdControlDealt_per_minute"] =  gamedata["team_totalTimeCrowdControlDealt"] / gamedata["gameMinute"]

#분당 팀의 시야 점수
gamedata["team_visionScore_per_minute"] = gamedata["team_visionScore"] / gamedata["gameMinute"]

#분당 팀의 골드 획득량 - 제외됨
#gamedata["team_goldEarned_per_minute"] = gamedata["team_goldEarned"] / gamedata["gameMinute"]

#분당 팀의 총 오브젝트 스코어량 - 제외됨
#gamedata["team_object_per_minute"] = gamedata["team_Object"] / gamedata["gameMinute"]

#분당 팀의 타워 파괴량
gamedata["towerKills_per_minute"] = gamedata["towerKills"] / gamedata["gameMinute"]

#분당 팀의 억제기 파괴량
gamedata["inhibitorKills_per_minute"] = gamedata["inhibitorKills"] /gamedata["gameMinute"]

#분당 팀의 바론 처치량
gamedata["baronKills_per_minute"] = gamedata["baronKills"] / gamedata["gameMinute"]

#분당 팀의 드래곤 처치량
gamedata["dragonKills_per_minute"] = gamedata["dragonKills"] / gamedata["gameMinute"]

#분당 팀의 전령 처치량
gamedata["riftHeraldKills_per_minute"] = gamedata["riftHeraldKills"] / gamedata["gameMinute"]

In [ ]:
#ex) 15분에 타워 10개 파괴 ->
10/15

In [ ]:
#ex) 30분에 타워 10개 파괴 ->
10/30

게임 시간에 맞춰 가중치가 반영된다.

In [ ]:
#inf 값 검사용
np.where(gamedata.values >= np.finfo(np.float64).max)

In [ ]:
#승패에 영향이 가는 상관계수 분석
gamedata.corr()["win"].sort_values()

새로 추가한 분당 성장률 지표의 상관계수가 이전의 팀 총합 지표보다 더욱 상승하였다.

## One hot Encoding  / 원 핫 인코딩

회귀 모델은 범주형 변수를 이해하지 못하므로 원 핫 인코딩을 적용해줌

In [ ]:
gamedata.drop("gameId",axis=1,inplace=True)

gamedata["time_bin"] = gamedata["time_bin"].astype("object")
gamedata = pd.get_dummies(gamedata)

## Feature Selection / 특성 선택

추가했던 칼럼들과 기존의 변수간의 관계를 살펴보고 다중공선성 제거, 유용한 칼럼들만 남김

특성 선택 기법에는 자동과 수동이 있는데, 굳이 수동적인 방법을 택한 이유는 향후 분석을 위해서이다.

1. PCA - 제외함

주성분 추출로, 자동으로 다중공선성을 제거하고 학습에 유용한 칼럼으로 차원축소되지만 해석이 어렵다.

2. RFE - 제외함

모든 변수를 우선 다 포함시킨 후 반복해서 학습을 진행하면서 중요도가 낮은 변수를 하나씩 제거하여 선택. 하지만 내부가 블랙박스이다.

2. Correlation Analysis

상관계수 분석을 통해, 각 변수간의 상관관계를 살피고 높은 칼럼들을 제거한다.

3. VIF

분산 인플레이션 계수 분석을 통해, 의존적인 칼럼들을 제거한다.

In [ ]:
import seaborn as sns    
plt.figure(figsize= (20, 10))
sns.heatmap(gamedata.corr())

개인 기록 칼럼들끼리 선형관계를 띄고 있다. 명백한 다중공선성 문제가 생길 것이라 예상.

In [ ]:
personal_col = ['kills1', 'kills2', 'kills3', 'kills4', 'kills5','deaths1', 'deaths2', 'deaths3', 'deaths4', 'deaths5','totalDamageDealtToChampions1', 'totalDamageDealtToChampions2',
       'totalDamageDealtToChampions3', 'totalDamageDealtToChampions4',
       'totalDamageDealtToChampions5','goldEarned1', 'goldEarned2',
       'goldEarned3', 'goldEarned4', 'goldEarned5','visionScore1',
       'visionScore2', 'visionScore3', 'visionScore4', 'visionScore5',
       'totalTimeCrowdControlDealt1', 'totalTimeCrowdControlDealt2',
       'totalTimeCrowdControlDealt3', 'totalTimeCrowdControlDealt4',
       'totalTimeCrowdControlDealt5']

gamedata.drop(personal_col,axis=1,inplace=True)

In [ ]:
import seaborn as sns    
plt.figure(figsize= (20, 10))
sns.heatmap(gamedata.corr())

팀 종합지표와 분당 팀 성장률이 높은 상관관계를 보이기 때문에 둘 중 하나는 제거해야함.

In [ ]:
gamedata.columns

In [ ]:
team_col = ['towerKills','inhibitorKills','baronKills','dragonKills','riftHeraldKills','team_kills', 'team_deaths','team_totalDamageDealtToChampions', 
            'team_totalTimeCrowdControlDealt','team_visionScore','team_K/D']

gamedata.drop(team_col,axis=1,inplace=True)

#오브젝트 갯수를 모두 합한 이 칼럼은 중복적인 요소가 많아 제거
#gamedata.drop("team_object_per_minute",axis=1,inplace=True)

#가설에서 직관적인 지표를 피드백해야 하고, 각 지표가 올라갈수록 골드량이 많아지는 것은 당연하기 때문에 해당 칼럼은 가설을 검정하기에 좋지 않음.
#gamedata.drop("team_goldEarned_per_minute",axis=1,inplace=True)

In [ ]:
import seaborn as sns    
plt.figure(figsize= (20, 10))
sns.heatmap(gamedata.corr())

독립변수간 상관관계가 많이 제거되었다.

In [ ]:
gamedata.drop(["gameDuration","gameMinute"],axis=1,inplace=True)

In [ ]:
#승패에 영향이 가는 상관계수 분석
gamedata.corr()["win"].sort_values()

In [ ]:
gamedata

In [ ]:
#VIF 분석으로 최종 검사 , 10 이상일 경우 다중공선성 문제가 생길 수 있음.
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    gamedata.values, i) for i in range(gamedata.shape[1])]
vif["features"] = gamedata.columns
vif

time_bin은 one hot encoding 된 항목으로 서로 상관관계가 있는것이 당연하다. 하지만 전부 나타내는 뜻이 다르므로 그대로 둔다.

# Modeling / 머신러닝 모델 구축

### Logistic Regression 로지스틱 회귀

모델을 구축하기 전, 마지막으로 로지스틱 회귀분석을 시행하여 Input 데이터셋을 분석한다.

In [ ]:
gamedata.shape

In [ ]:
x_data = gamedata.drop("win",axis=1)
y_data = gamedata["win"]

In [ ]:
x_data.info()

In [ ]:
#사용하는 Input column
use_col = list(x_data.columns)
use_col

In [ ]:
import statsmodels.api as sm

logit = sm.Logit(y_data,x_data) #로지스틱 회귀분석 시행
result = logit.fit()
result.summary2()

모든 변수의 P - Value가 0에 가까우므로 통계적으로 유의미한 변수들이 적합되었고, 아무런 튜닝 없이 단순 로지스틱 회귀로 R-squared(결정계수) 가 0.91에 달함. 

### Logistic Regression Analytics / 로지스틱 회귀 분석

선형회귀는 그대로 회귀계수가 타켓변수에 영향을 미치지만, 로지스틱 회귀는 약간 다르다.

로지스틱 회귀의 오즈는 0일 확률 대비 1일 확률을 나타내므로 회귀계수에 Expotential ^ x 를 적용하면 1이 될 확률에 미치는 영향이 계산된다.

In [ ]:
#모든 변수를 하나의 모델에서 해석
for i in range(len(result.params)):
    print('다른 변수가 변하지 않을 때, {} 이 한단위 상승하면 승리할 확률이 {:.5f} 배 증가한다.\n'.format(result.params.keys()[i],np.exp(result.params.values[i])))

하지만 데이터들의 스케일이 달라 가중치가 지나치게 크고, 작고 문제가 발생했다.

In [ ]:
from sklearn.model_selection import train_test_split

#훈련셋, 테스트셋 계층 샘플링 - 편향 방지
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7, stratify=y_data) 

## Data Scaling / 데이터 스케일링

데이터의 학습 속도, 모델의 안정성을 증가시킨다.

Scikit-learn이 제공하는 데이터 스케일링 클래스

1. StandardScaler - 채택

평균이 0과 표준편차가 1이 되도록 변환

2. RobustScaler

중앙값(median)이 0, IQR(interquartile range)이 1이 되도록 변환.

3. MinMaxScaler

최대값이 각각 1, 최소값이 0이 되도록 변환

4. MaxAbsScaler

 0을 기준으로 절대값이 가장 큰 수가 1또는 -1이 되도록 변환

In [ ]:
#각 feature의 평균을 0, 분산을 1로 변경 - 특성들을 모두 동일한 스케일로 반영

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_train = ss.transform(x_train)
x_test = ss.transform(x_test)

In [ ]:
import statsmodels.api as sm

logit = sm.Logit(y_train,x_train) #로지스틱 회귀분석 시행
result = logit.fit()
result.summary2()

In [ ]:
#모든 변수를 하나의 모델에서 해석
for i in range(len(result.params)):
    print('다른 변수가 변하지 않을 때, {} 이 1 상승하면 승리할 확률이 {:.5f} 배 증가한다.\n'.format(use_col[i],np.exp(result.params.values[i])))

특성이 조절되고 이전보다 더 적절한 값이 출력된다. 하지만 아직 특정 칼럼의 가중치가 조금 높으므로 규제를 추가한다.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
#그리드 서치로 하이퍼파라미터 튜닝, 계층별 10차 교차 검증 수행
param_grid = {"C" : [0.001,0.01,0.1,1,10,10,100]} #규제의 강도만 조절하고, 방식은 바꾸지 않는다. 조금이라도 승패에 영향이 가는 변수를 모두 반영하기 위해 L2(릿지)norm을 적용.
grid_search = GridSearchCV(LogisticRegression(),param_grid,cv=10,return_train_score=True,n_jobs=-1)

grid_search.fit(x_train,y_train)

In [ ]:
coef = pd.DataFrame(data={"coef":grid_search.best_estimator_.coef_.tolist()[0],"col":use_col})
coef

In [ ]:
for i in range(len(coef)):
    print('다른 변수가 변하지 않을 때, {} 이 1 상승하면 승리할 확률이 {:.5f} 배 증가한다.\n'.format(use_col[i],np.exp(coef.iloc[i,0])))

K/D와 towerKill은 게임 구조상 분당 1개씩 증가할 수 없는 수치이지만, 만약 가능하다면 승리 확률이 어마어마하게 올라간다.
time_bin을 보면 예상대로 시간이 갈수록 승리에 미치는 영향이 줄어든다는 것을 알 수 있다.

하지만 로지스틱 회귀는 공선성 영향을 크게 받기 때문에(중요한 칼럼만 가중치가 커짐) 해당 회귀 분석결과를 그대로 신뢰할 수는 없다.
또한 로지스틱 회귀는 안좋게 될 확률(0이 될 확률)을 모르기 때문에 패배 확률 자체가 낮다면 의미가 없다.

In [ ]:
print("최적 파라미터 : {}".format(grid_search.best_params_))

print("훈련세트 정확도(교차 검증 정확도 평균) : {}".format(grid_search.best_score_))

print(grid_search.best_estimator_)

In [ ]:
#결과표시
np.set_printoptions(suppress=True)

pred_y_test = grid_search.predict(x_test)
proba_y_test = grid_search.predict_proba(x_test)
display(pred_y_test,proba_y_test)

In [ ]:
print("테스트세트 정확도 : {}".format(grid_search.score(x_test,y_test)))
log_cls_score = grid_search.score(x_test,y_test)

In [ ]:
#f1 스코어
from sklearn.metrics import f1_score
score = f1_score(y_test,pred_y_test)
print("테스트세트 f1 점수 : {}".format(score))

단순한 로지스틱 회귀에 스케일링, 규제, 교차검증을 적용한 모델의 테스트세트 정확도는 97.6%에 달한다.

## Model Selection /  모델 선택

로지스틱 이외에도 이진 분류에 적합한 많은 모델들이 있는데, 그 중에 몇가지를 선정하여 실험

모델의 평가 지표는 Accuracy(정확도) 로 타켓 클래스가 균형을 이루고 있기 때문.

> 선정한 모델

* Random Forest 랜덤 포레스트

단일 결정 트리의 단점을 보완하고, 매개변수 튜닝 없이도 강력한 성능을 보이는 모델이며, 특성 중요도를 제공하여 채택.

* Catboost 캣 부스트

데이터셋의 절반은 카테고리형으로 구성되어 있고, 그래디언트 부스팅 모델 중 매개변수 튜닝 없이도 강력한 성능을 보이는 모델이므로 채택.

* XGBoost Extream Gradient boosting

기존 그래디언트 부스팅 모델의 단점을 보완하고 빠른 학습을 한다. 매개변수에 민감하지만 GPU 병렬연산을 이용한 그리드 서치를 지원하여 채택.

============================================================================================================================

> 고려한 모델

* KNN 최근접 이웃 알고리즘

해당 데이터셋에서 이웃간 거리를 이용한 분류가 어렵고, 데이터셋이 크고 적절한 이웃 갯수를 찾는 것이 어려우므로 기각.

* SVM 서포트 벡터 머신

고차원의 해당 데이터셋에서 잘 맞을거라 생각하지만, 분석이 어려워서 기각.

* Naive bayes 나이브 베이즈

선형 모델보다 빠른 학습이 가능하고 대용량, 고차원 데이터에 적합하지만 모든 특징들이 동등하고 독립적이라 가정하는 모델이라 어느정도 독립변수간 상관관계가 존재하는 경기기록에는 맞지 않을거라 생각.

* LightGBM

기존 그래디언트 부스팅 모델의 단점을 보완하고 빠른 학습을 하지만, 매개변수 튜닝 시, XGBoost가 상대적으로 좋은 결과를 가져와서 기각.

* Neural Network / Deep Learning 신경망 / 딥러닝

매우 복잡한 모델을 설계할 수 있고 대용량, 고차원 데이터에 적합하지만 SWM과 마찬가지로 하이퍼 파라미터 튜닝이 복잡함.

In [ ]:
#실제 모델들과 비교하기 위한 무작위 랜덤모델
from sklearn.dummy import DummyClassifier
dm = DummyClassifier(strategy="stratified").fit(x_train,y_train)
dm.score(x_test,y_test)

랜덤선택모델은 0.49 ~0.5 의 정확도를 보이므로, 데이터에 불균형이 존재하지 않음.

### Random Forest Classifier/ 랜덤 포레스트 분류기 (결정 트리 앙상블)

In [ ]:
# %%time
# from sklearn.ensemble import RandomForestClassifier
# #그리드 서치로 하이퍼파라미터 튜닝, 계층별 5차 교차 검증 수행
# param_grid = {"n_estimators" : [10,50,100,500],
#              "criterion": ["entropy","gini"]}

# rf = RandomForestClassifier(random_state=7,n_jobs=-1)
# grid_search = GridSearchCV(rf,param_grid,cv=5,return_train_score=True)

# grid_search.fit(x_train,y_train)

# print("최적 파라미터 : {}".format(grid_search.best_params_))

# print("훈련세트 정확도(교차 검증 정확도 평균) : {}".format(grid_search.best_score_))

# print(grid_search.best_estimator_)

# print("테스트세트 정확도 : {}".format(grid_search.score(x_test,y_test)))
# rf_cls_score = grid_search.score(x_test,y_test)

# result = pd.DataFrame(grid_search.cv_results_)
# result

#최적 파라미터 : {'criterion': 'entropy', 'n_estimators': 100}
#훈련세트 정확도(교차 검증 정확도 평균) : 0.9791780438283274

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,criterion="entropy",random_state=7,n_jobs=-1)
rf.fit(x_train,y_train)

In [ ]:
print("훈련세트 정확도 : {}".format(rf.score(x_train,y_train)))

print("테스트세트 정확도 : {}".format(rf.score(x_test,y_test)))

Random Forest에 파라미터 튜닝을 적용한 모델은 정확도는 97.8에 달함.

In [ ]:
#특성 중요도 분석
import seaborn as sns

plt.figure(figsize=(20,12))
feat_imp = {'col' : x_data.columns,
            'importances' : rf.feature_importances_}

feat_imp = pd.DataFrame(feat_imp).sort_values(by = 'importances', ascending = False)

sns.barplot(y = feat_imp['col'] ,
            x = feat_imp['importances'])

로지스틱 회귀와 마찬가지로 게임의 승패를 판가름하는 중요한 요소는 분당 팀의 K/D, 분당 팀의 타워 파괴량으로 나타났다.(롤은 타워를 부숴야 이길 수 있는 게임이므로)

회귀분석에서 잡아내지 못했던 것은, 분당 킬보다 분당 데스가 더 중요하게 나타났다. 이는 모델이 해석한 결과, 킬을 많이하는 것보다는 데스를 적게하는것이 중요하다는 것이다.

또한 EDA에서 분석한대로 대부분의 first(선취) 관련 칼럼들은 중요도가 낮은 반면, first_inhibitor은 킬데스 다음으로 중요하다고 한다. 이는 게임시에 목표 오브젝트는 억제기가 최우선이다, 라는 통찰을 제공한다.



### Catboost / 카테고리 부스트 (그라디언트 부스팅)

catboost 모델은 xgboost,lgbm 과 달리 트리의 다형성과 오버피팅 문제를 내부 알고리즘이 자체적으로 지원하여 파라미터 튜닝이 크게 필요없음.

In [ ]:
from catboost import CatBoostClassifier

cbr = CatBoostClassifier(verbose=50,task_type="GPU")
cbr.fit(x_train,y_train)

In [ ]:
print("훈련세트 정확도 : {}".format(cbr.score(x_train,y_train)))

print("테스트세트 정확도 : {}".format(cbr.score(x_test,y_test)))

튜닝을 하지 않았지만, 기본 옵션의 GPU Catboost 정확도는 97.9 - 98.0 %에 달한다.

In [ ]:
#특성 중요도 분석
import seaborn as sns

f,ax = plt.subplots(1,2,figsize=(20,12))
rf_imp = {'col' : x_data.columns,
            'rf_importances' : rf.feature_importances_}

rf_imp = pd.DataFrame(rf_imp).sort_values(by = 'rf_importances', ascending = False)

sns.barplot(y = rf_imp['col'] ,
            x = rf_imp['rf_importances'],ax=ax[0])

cbr_imp = {'col' : x_data.columns,
            'cbr_importances' : cbr.feature_importances_}

cbr_imp = pd.DataFrame(cbr_imp).sort_values(by = 'cbr_importances', ascending = False)

sns.barplot(y = cbr_imp['col'] ,
            x = cbr_imp['cbr_importances'],ax=ax[1])

캣부스트 모델에서는 랜덤포레스트보다, 데스를 더 중요시하고, 팀의 총데미지의 중요도가 훨씬 높아졋다. 또한 오브젝트들의 칼럼 중요도가 팀의 기록보다 더 중요한 형태로 바뀌었다.

이는 카테고리 칼럼을 잘 처리하는 캣포레스트 특성이 반영되어, 카테고리적인 느낌을 띄는 오브젝트 킬 칼럼들의 중요도가 높아진 것으로 보인다.

### XGBoost / Extreme Gradient Boosting (그라디언트 부스팅)

In [ ]:
# from xgboost import XGBClassifier

# xgb = XGBClassifier()

# param_grid = {'objective':['binary:logistic'],
#               'learning_rate': [0.025], #catboost에서 측정된 적절 eta
#               'max_depth': [2,3,5],
#               'min_child_weight': [1,5,10], #최소 가중치 합 : 높게하면 언더피팅
#               'colsample_bytree': [0.7],
#               'n_estimators': [10,50,100,1000],
#               'seed': [7]}

# grid_search = GridSearchCV(xgb, param_grid, n_jobs=-1, 
#                    cv=5, 
#                    verbose=2, refit=True)

# grid_search.fit(x_train, y_train)

#최적 파라미터 : {'objective':['binary:logistic'],
#               'learning_rate': [0.025], #catboost에서 측정된 적절 eta
#               'max_depth': [3],
#               'min_child_weight': [10], #최소 가중치 합 : 높게하면 언더피팅
#               'colsample_bytree': [0.7],
#               'n_estimators': [1000],
#               'seed': [7]}
#훈련세트 정확도(교차 검증 정확도 평균) : 0.9795003920690042

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(
    objective='binary:logistic',
    max_depth=3,
    n_estimators=1000,
    min_child_weight=10, 
    colsample_bytree=0.7, 
    eta=0.025,
    nthread=-1,
    seed=7)

xgb.fit(x_train, y_train, verbose=True)

In [ ]:
print("훈련세트 정확도 : {}".format(xgb.score(x_train,y_train)))

print("테스트세트 정확도 : {}".format(xgb.score(x_test,y_test)))

In [ ]:
xgb.feature_importances_

In [ ]:
#특성 중요도 분석
import seaborn as sns

f,ax = plt.subplots(1,3,figsize=(20,12))
rf_imp = {'col' : x_data.columns,
            'rf_importances' : rf.feature_importances_}

rf_imp = pd.DataFrame(rf_imp).sort_values(by = 'rf_importances', ascending = False)

sns.barplot(y = rf_imp['col'] ,
            x = rf_imp['rf_importances'],ax=ax[0])

cbr_imp = {'col' : x_data.columns,
            'cbr_importances' : cbr.feature_importances_}

cbr_imp = pd.DataFrame(cbr_imp).sort_values(by = 'cbr_importances', ascending = False)

sns.barplot(y = cbr_imp['col'] ,
            x = cbr_imp['cbr_importances'],ax=ax[1])

xgb_imp = {'col' : x_data.columns,
            'xgb_importances' : xgb.feature_importances_}

xgb_imp = pd.DataFrame(xgb_imp).sort_values(by = 'xgb_importances', ascending = False)

sns.barplot(y = xgb_imp['col'] ,
            x = xgb_imp['xgb_importances'],ax=ax[2])

흥미로운 결과가 나타났다. XGB 모델은 time_bin의 중요도가 대폭 상승했는데, 그 중에서도 20분이내, 30분이후를 나타내는 지표가 대폭 상승했다. 

시간대 앞에 inhibitorKills와 tower킬이 있는 것을 봐선, 시계열적인 학습을 한게 아닐까 추측된다.

뿐만 아니라, 다른 모델들에서도 time_bin_4가 어느정도의 중요성을 띄는 것이 보이는데 EDA와 회귀분석에서 추측한 결과로는, 게임시간이 길어질수록 승패가 불투명해질거라 생각했는데

모델들이 해석한 결과는 게임시간이 길어질수록 승패가 명확해진다는 것을 의미한다.

XGB모델은 그 중에서도 k/d 가 높고, 억제기 파괴량이 높고, 포탑 파괴량이 높을때, 시간대가 30분 이후나, 20분 이전이라면 승패가 명확하다는 것을 학습한 것 같다.

### Model Save & Load

앞으로 실제 데이터의 예측에 있어선 time_bin_4, time_bin_1일때에 대해서 XGB모델을 적용하고, time_bin_2,time_3 일때 전체적인 예측력이 높은 Catboost 모델을 사용할 것이다.

In [ ]:
cbr.save_model("LOL_predict_cbr.cbm")

In [ ]:
xgb.save_model("LOL_predict_xgb.bst")

In [ ]:
cbr = CatBoostClassifier()
cbr.load_model("LOL_predict_cbr.cbm")

In [ ]:
xgb = XGBClassifier({'nthread': 4})
xgb.load_model('LOL_predict_xgb.bst')